In [6]:
!pip install opencv-python
import cv2
import numpy as np
from matplotlib import pyplot
from keras.datasets import fashion_mnist
from keras.utils import to_categorical
from keras.models import Sequential
from keras.layers import Conv2D
from keras.layers import MaxPooling2D
from keras.layers import Dense
from keras.layers import Dropout
from keras.layers import Flatten
from keras.optimizers import SGD
from keras.applications.vgg19 import VGG19
from keras.applications.vgg19 import preprocess_input, decode_predictions

# load dataset
(trainX, trainY), (testX, testY) = fashion_mnist.load_data()
# reshape dataset to have a single channel
trainX = trainX.reshape((trainX.shape[0], 28, 28, 1))
testX = testX.reshape((testX.shape[0], 28, 28, 1))

trainX = [cv2.cvtColor(cv2.resize(i, (48, 48)), cv2.COLOR_GRAY2RGB) for i in trainX]
trainX = np.concatenate([arr[np.newaxis] for arr in trainX]).astype('float32')
testX = [cv2.cvtColor(cv2.resize(i, (48, 48)), cv2.COLOR_GRAY2RGB) for i in testX]
testX = np.concatenate([arr[np.newaxis] for arr in testX]).astype('float32')

# one hot encode target values
trainY = to_categorical(trainY)
testY = to_categorical(testY)

train_norm = trainX / 255.0
test_norm = testX / 255.0

#修改成VGG19 model
vgg19_model = VGG19(weights='imagenet', include_top=False, input_shape=(48, 48, 3))

model = Sequential()
model.add(vgg19_model)
model.add(Flatten())
model.add(Dense(128, activation='relu', kernel_initializer='he_uniform'))
model.add(Dropout(0.5))
model.add(Dense(64, activation='relu', kernel_initializer='he_uniform'))
model.add(Dropout(0.5))
model.add(Dense(10, activation='softmax'))

# compile model
opt = SGD(lr=0.01, momentum=0.9)
model.compile(optimizer=opt, loss='categorical_crossentropy', metrics=['accuracy'])

# evaluate a model using k-fold cross-validation
model.fit(train_norm, trainY, epochs=10, batch_size=32, validation_data=(test_norm, testY), verbose=1)

test_loss, test_acc = model.evaluate(test_norm, testY)
print('Test accuracy: {:2.1f}%'.format(test_acc*100))

model.summary()

Epoch 1/10
1875/1875 [==============================] - 3596s 2s/step - loss: 0.9741 - accuracy: 0.6115 - val_loss: 0.5358 - val_accuracy: 0.7919
Epoch 2/10
1875/1875 [==============================] - 3225s 2s/step - loss: 0.4991 - accuracy: 0.8232 - val_loss: 0.3670 - val_accuracy: 0.8814
Epoch 3/10
1875/1875 [==============================] - 3385s 2s/step - loss: 0.3833 - accuracy: 0.8784 - val_loss: 0.3065 - val_accuracy: 0.9023
Epoch 4/10
1875/1875 [==============================] - 3409s 2s/step - loss: 0.3073 - accuracy: 0.9023 - val_loss: 0.2682 - val_accuracy: 0.9103
Epoch 5/10
1875/1875 [==============================] - 3346s 2s/step - loss: 0.2895 - accuracy: 0.9086 - val_loss: 0.2612 - val_accuracy: 0.9165
Epoch 6/10
1875/1875 [==============================] - 3243s 2s/step - loss: 0.2509 - accuracy: 0.9212 - val_loss: 0.2592 - val_accuracy: 0.9229
Epoch 7/10
1875/1875 [==============================] - 3248s 2s/step - loss: 0.2314 - accuracy: 0.9255 - val_loss: 0.2575 -